
# Reasons to Choose Houston,TX.

For this project I choose Houston,Texas. I choose this city becouse I had lived there for several years during my study at University of Houston .So I'm familier with the city, also crating mapping and wringling the street data bring some great memories.

https://www.openstreetmap.org/node/27526178



### Objective

- Assess the quality of the data for validity, accuracy, completeness, consistency and uniformity.
- Parse and gather data from popular file formats such as .csv, .json, .xml, and .html
- Process data from multiple files or very large files that can be cleaned programmatically.
- Learn how to store, query, and aggregate data using SQL.


### Data Assessment

- Abbreviations of street types for example "ST", "AV"،"DR" etc.
- All lowercase letters for example "avenue" instead of "Avenue".
- iter through all zip codes, collect all the zip codes that does not start with 15 since pittsburgh PA zip codes with 15

 ### Auditing and cleaning The dataset

In [3]:
#imports libraries

import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint
import csv
import codecs
import schema
import sqlite3


In [ ]:
#osm file
OSMFILE = "houston_texas.osm"

# Tag Types

In [ ]:

import xml.etree.cElementTree as ET
import pprint


def count_tags(filename):
    tags = {}
    for event, elem in ET.iterparse(filename):
        if elem.tag in tags: 
            tags[elem.tag] += 1
        else:
            tags[elem.tag] = 1
    return tags
    
"""
return tag type as well as count for each tag.
"""
pprint.pprint(count_tags(OSMFILE))

#  k Tags

In [ ]:

"""
Count multiple patterns in the tags
"""
lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')



In [ ]:
def key_type(element, keys):
    if element.tag == "tag":
        
        if lower.search(element.attrib['k']):
            keys['lower'] += 1
        elif lower_colon.search(element.attrib['k']):
            keys['lower_colon'] += 1
        elif problemchars.search(element.attrib['k']):
            keys['problemchars'] += 1
        else:
            keys['other'] += 1

    return keys
    
def process_map(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)

    return keys

In [ ]:
pprint.pprint(process_map(OSMFILE))

## Unique users



In [ ]:
"""
The unique users have contributed to the data set

The function process_map should return a set of unique user IDs
"""

def get_user(element):
    return


def process_map(filename):
    users = set()
    for _, element in ET.iterparse(filename):
        if "uid" in element.attrib:
            users.add(element.attrib["uid"])
    return users

In [ ]:
# printing out the total number of unique users

print(len(process_map(OSMFILE)))


#  Cleaning street names


In [ ]:
# Regular expression compilers

OSM_f = "houston_texas.osm"

regex = re.compile(r'\b\S+\.?$', re.IGNORECASE)
regexc = re.compile(r'^[NSEW]\b\.?', re.IGNORECASE)  

expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons", "Bend", "Chase", "Circle", "Cove", "Crossing", "Hill",
            "Hollow", "Loop", "Park", "Pass", "Overlook", "Path", "Plaza", "Point", "Ridge", "Row",
            "Run", "Terrace", "Walk", "Trace", "View", "Vista"]

mapping = { "St": "Street",
            "St.": "Street",
            "street": "Street",
            "Ave": "Avenue",
            "Ave.": "Avenue",
            "Blvd": "Boulevard",
            "Blvd.": "Boulevard",
            "Boulavard": "Boulevard",
            "Rd": "Road",
            "Rd.": "Road",
            "RD": "Road",
            "Pl": "Place",
            "Pl.": "Place",
            "PKWY": "Parkway",
            "Pkwy": "Parkway",
            "Ln": "Lane",
            "Ln.": "Lane",
            "Dr": "Drive",
            "Dr.": "Drive"
            }
# Adding street names in dictionary by type
# Takes two arguments: dictionary and string, whene string didn't match pattern it will adds it to dictionary. 
def audit_street_type(street_types, street_name, regex, expected):  
    m = regex.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)

In [ ]:
#Checks if attribute is a street 
#Args:
#elem: Element from OpenStreetMap data
#Returns:
#True if a valid street name. False otherwise.

def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")

In [ ]:
#parsing and auditing of the street names.
def audit(osm_file, regex):
    osm_file = open(OSM_f, "r")    
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'], regex, expected)

    return street_types

pprint.pprint(dict(audit(OSM_f, regex))) # print the existing names

#### Update street name by replaceing unexpected street names with better names.

In [ ]:
# return the updated names
def update_name(name, mapping, regex):  
    m = regex.search(name)
    if m:
        street_type = m.group()
        if street_type in mapping:
            name = re.sub(regex, mapping[street_type], name)

    return name

update_street = audit(OSM_f, regex) 

# print the updated names
for street_type, ways in update_street.iteritems():
    for name in ways:
        better_name = update_name(name, mapping, regex)
        print name, "=>", better_name 

### find and replace the abbreviated cardinal points, North, South, East, and West

In [ ]:
cardinal_points = audit(OSM_f, regexc)

pprint.pprint(dict(cardinal_points))  

In [ ]:
cardinal_mapping = {'N'  : 'North', 
                    'N.' : 'North',
                    'E.' : 'East',
                    'E'  : 'East',
                    'W'  : 'West',
                    'W.' : 'West',
                    'S'  : 'South',
                    'S.' : 'South'}

for cardinal_points, ways in cardinal_points.iteritems():  
    if cardinal_points in cardinal_mapping:
        for name in ways:
            better_name = update_name(name, mapping, regex)
            nice_name   = update_name(better_name, cardinal_mapping, regexc)
            print name, "=>", better_name, "=>", nice_name

# Postal Codes
After cleaning the street names, now we are going to audite and clean the postal code for Houston area.Postal codes for Houston is 5-numeric starting with 77. So for  consistency we are going to clean up al zip code that does not Houston area zip codes.

In [ ]:
def audit_zipcodes(OSMFILE):
    # iter through all zip codes, collect all the zip codes that does not start with 77
    
    zip_codes = {}
    for event, elem in ET.iterparse(OSMFILE, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if tag.attrib['k'] == "addr:postcode" and not tag.attrib['v'].startswith('77'):
                    if tag.attrib['v'] not in zip_codes:
                        zip_codes[tag.attrib['v']] = 1
                    else:
                        zip_codes[tag.attrib['v']] += 1
    return zip_codes

zipcodes = audit_zipcodes(OSMFILE)
for zipcode in zipcodes:
    print (zipcode, zipcodes[zipcode])